In [91]:
import pandas as pd
import numpy as np
import time
import re
from sklearn.metrics.pairwise import cosine_similarity

def year_interval(filter_value,cols):
    filter_value_1 = filter_value.dropna(subset = cols) #filter_value.sort_values(by=['apply_date'],ascending=True)
    return (filter_value_1['申请日'].max()-filter_value_1['申请日'].min()).days//365+1

def process_ipc(filter_patent):
    ipcs_patent = filter_patent['IPC'].str.split(';').explode().str.strip().str.extract('(\w+)/')[0].str[:3]
    return 1.0 - (ipcs_patent.value_counts(normalize=True)**2).sum()

def calculate_metrics(filter_value):
    metrics = {}
    
    # 申请号的唯一数量
    metrics['Application_num'] = filter_value['申请号'].nunique()
    
    # 去重后的 IPC 主分类数量
    metrics['IPC主分类_去重'] = filter_value['IPC主分类'].dropna().map(lambda x: x[:3]).nunique()
    
    # IPC 主分类的平均值
    metrics['IPC主分类_平均'] = metrics['IPC主分类_去重'] / year_interval(filter_value, 'IPC主分类')

    # 计算 IPC 的去重数量和平均值
    ipcs = filter_value['IPC'].str.split(';').explode().str.strip()
    metrics['IPC_个数_去重'] = ipcs.nunique()
    metrics['IPC_个数_平均'] = metrics['IPC_个数_去重'] / year_interval(filter_value, 'IPC')

    # 被引证次数和引证数量
    metrics['被引证次数'] = filter_value['被引证次数'].sum()
    metrics['引证数量'] = filter_value['引证次数'].sum()
    
    # 引证专利的去重数量和平均值
    citing = filter_value['引证专利'].str.split(';').explode().str.strip()
    metrics['引证数量_去重'] = citing.nunique()
    metrics['引证数量_平均'] = metrics['引证数量_去重'] / year_interval(filter_value, '引证专利')

    # 简单同族专利相关统计
    metrics['简单同族个数'] = filter_value['简单同族个数'].sum()
    cognate = filter_value['简单同族'].str.split(';').explode().str.strip()
    metrics['简单同族_去重'] = cognate.nunique()
    metrics['简单同族美日欧_比例'] = cognate.str[:2].isin(['US', 'JP', 'EP']).sum() / cognate.shape[0]

    # 发明人数量
    metrics['发明人数量'] = filter_value['发明人数量'].sum()
    metrics['发明人数量_去重'] = filter_value['发明人'].str.split(';').explode().str.strip().nunique()
    metrics['发明人数量_平均'] = metrics['发明人数量_去重'] / year_interval(filter_value, '发明人')

    # 权利要求数量和引证科技文献数量
    metrics['权利要求数量'] = filter_value['权利要求数量'].sum()
    metrics['引证科技文献数量'] = filter_value['引证科技文献'].count()  # 80%空,格式不统一
    
    # 有效专利数量和有效比例
    metrics['有效数量'] = (filter_value['专利有效性'] == '有效').sum()
    metrics['有效_比例'] = metrics['有效数量'] / filter_value.shape[0]

    # 发明授权和发明申请专利数量
    metrics['发明授权'] = (filter_value['专利类型'] == '发明授权').sum()
    metrics['发明申请'] = (filter_value['专利类型'] == '发明申请').sum()

    metrics['实用新型']=(filter_value['专利类型'] == '实用新型').sum()
    metrics['外观设计']=(filter_value['专利类型'] == '外观设计').sum()

    copatent = (filter_value['申请人'].str.count(';') + 1 > 1)
    metrics['invention'] = (~copatent & ((filter_value['专利类型'] == '发明授权') | (filter_value['专利类型'] == '发明申请'))).sum()
    metrics['shiyong'] = (~copatent & (filter_value['专利类型'] == '实用新型')).sum()
    metrics['waiguan'] = (~copatent & (filter_value['专利类型'] == '外观设计')).sum()
    
    # 联合申请
    metrics['copatent_invention'] = (copatent & ((filter_value['专利类型'] == '发明授权') | (filter_value['专利类型'] == '发明申请'))).sum()
    metrics['copatent_shiyong'] = (copatent & (filter_value['专利类型'] == '实用新型')).sum()
    metrics['copatent_waiguan'] = (copatent & (filter_value['专利类型'] == '外观设计')).sum()

    return metrics

chinese_to_arabic = {
    '零': '0', '一': '1', '二': '2', '三': '3', '四': '4',
    '五': '5', '六': '6', '七': '7', '八': '8', '九': '9'
}

# 1. 剔除异常符号
def remove_unwanted_symbols(name):
    # 定义允许的字符：中文、英文、数字和常见符号（不想删除的符号）
    allowed_characters = r'[^\w\s\(\)（）【】「」『』]'
    # 删除所有不允许的符号
    return re.sub(allowed_characters, '', name)

# 2. 将各种括号转换成英文括号
def convert_brackets(name):
    # 替换中文和全角括号为标准英文括号
    return name.replace('（', '(').replace('）', ')') \
               .replace('【', '[').replace('】', ']') \
               .replace('「', '[').replace('」', ']') \
               .replace('『', '[').replace('』', ']')

# 3. 将汉字数字替换成阿拉伯数字
def replace_chinese_numbers(name):
    for ch_num, ar_num in chinese_to_arabic.items():
        name = name.replace(ch_num, ar_num)
    return name

# 4. 检查并调整括号匹配
def fix_unbalanced_brackets(name):
    left_count = name.count('(')
    right_count = name.count(')')
    
    # 如果左括号比右括号多，补齐右括号
    if left_count > right_count:
        name += ')' * (left_count - right_count)
    # 如果右括号比左括号多，补齐左括号
    elif right_count > left_count:
        name = '(' * (right_count - left_count) + name
    
    return name

# 5. 移除前面的数字和数字开头的部分
def remove_leading_numbers(name): 
    return re.sub(r'^\d+.*?(?=\D)', '', name).strip()  # re.sub(r'^\d+.*? ', '', name) 移除开头的数字直到第一个非数字字符，并去掉前后空格

    
# 综合清洗函数
def clean_fullname(name):
    if not isinstance(name, str):
        return name
    name = remove_unwanted_symbols(name)  # 剔除异常符号
    name = convert_brackets(name)  # 将括号转换成英文括号
    name = replace_chinese_numbers(name)  # 替换汉字数字为阿拉伯数字
    name = fix_unbalanced_brackets(name)  # 调整不平衡的括号
    name = remove_leading_numbers(name)  # 移除前面的数字
    return name.upper() # 转换为大写并去掉前后空格

words_to_remove = [
    '有限', '责任', '股份', '公司', '厂', '省', '市', '区', '县',
    '回族自治区', '壮族自治区', '维吾尔自治区', '自治区'
]
# words_to_remove = [
#     '有限', '责任', '股份', '集团', '总公司', '分公司', '公司', '总院', '分院', '总部', '分部', '总厂', '厂', 
#     '省', '市', '区', '县', '回族自治区', '壮族自治区', '维吾尔自治区', '自治区'
# ]


# 1. 剔除特定词汇
def remove_specific_words(name):
    # 构建正则表达式，匹配列表中的词汇
    pattern = '|'.join(words_to_remove)
    # 替换这些词汇为空
    return re.sub(pattern, '', name)

# 2. 剔除括号和括号内的内容
def remove_brackets(name):
    # 删除所有括号和其中的内容
    return re.sub(r'\(.*?\)', '', name)

# 综合清洗得到企业简称
def clean_short_name(name):
    if not isinstance(name, str):
        return name
    name = remove_specific_words(name)  # 剔除特定词汇
    name = remove_brackets(name)  # 剔除括号及其中内容
    return name.strip()  # 去除首尾空格

In [43]:
patent_tax_gx=pd.read_csv('X:\Downloads\Patent\patent_tax_gx.csv')
patent_tax_gx.shape#1432848+29874,32

C:\Users\Prime\AppData\Local\Temp\ipykernel_215656\659431814.py:1: DtypeWarning: Columns (18,19,22) have mixed types. Specify dtype option on import or set low_memory=False.
  patent_tax_gx=pd.read_csv('X:\Downloads\Patent\patent_tax_gx.csv')


(1462722, 32)

In [44]:
patent_tax_gx_2=pd.read_csv('X:\Downloads\Patent\patent_tax_gx_2.csv')
patent_tax_gx=pd.concat([patent_tax_gx,patent_tax_gx_2])
patent_tax_gx.shape

(1475181, 32)

In [45]:
patent_tax_gx['申请日']=pd.to_datetime(patent_tax_gx['申请日'])
patent_tax_gx['公开（公告）日']=pd.to_datetime(patent_tax_gx['公开（公告）日'])
patent_tax_gx['申请日'].describe(),patent_tax_gx['公开（公告）日'].describe()

(count                          1475181
 mean     2015-03-20 13:18:03.564662016
 min                1986-05-27 00:00:00
 25%                2012-11-30 00:00:00
 50%                2015-12-25 00:00:00
 75%                2018-03-19 00:00:00
 max                2019-12-31 00:00:00
 Name: 申请日, dtype: object,
 count                          1475181
 mean     2016-07-07 09:58:04.047856640
 min                1988-09-21 00:00:00
 25%                2014-03-19 00:00:00
 50%                2017-03-08 00:00:00
 75%                2019-05-07 00:00:00
 max                2024-08-13 00:00:00
 Name: 公开（公告）日, dtype: object)

In [46]:
patent_tax_gx=patent_tax_gx.sort_values(['申请日','公开（公告）日']).drop_duplicates(['申请号','公开（公告）号'],keep='first').reset_index(drop=True)
patent_tax_gx.shape#1198466

(1208568, 32)

In [38]:
tax_gx_patent_bridge=pd.read_csv('data\\tax_gx_patent_bridge_2019.csv')
tax_gx_patent_bridge=patent_tax_gx_1[~patent_tax_gx_1['公开（公告）号'].isin(patent_tax_gx['公开（公告）号'])]
tax_gx_patent_bridge.to_csv('X:\Downloads\Patent\\tax_gx_patent_bridge_2019_2.csv')
tax_gx_patent_bridge.shape

(12603, 5)

In [20]:
tax_gx_patent_bridge=pd.read_csv('X:\Downloads\Patent\\tax_gx_patent_bridge_2408.csv',usecols=['申请号','企业简称'])
combined_tax_gx_companies = tax_gx_patent_bridge['企业简称'].replace('',pd.NA).dropna().unique()
len(tax_gx_patent_bridge),len(combined_tax_gx_companies),tax_gx_patent_bridge.申请号.nunique()

(1582358, 8451, 1067856)

In [48]:
# patent_applicants=patent_tax_gx['申请人'].str.split(';').explode().str.strip()
# patent_applicants_short=patent_applicants.apply(clean_fullname).apply(clean_short_name)
# mask=patent_applicants_short.isin(combined_tax_gx_companies)
# patent_all_7 = patent_tax_gx.loc[mask[mask].index]

patent_tax_gx['申请年']=patent_tax_gx['申请日'].dt.year
patent_tax_gx['企业名称']=patent_tax_gx['申请人'].str.split(';')

patent_co['申请人数量']=patent_co['申请人'].str.count(';')+1
patent_co=patent_tax_gx.explode(['企业名称'])
patent_co['企业名称']=patent_co['企业名称'].str.strip()
patent_co['企业简称']=patent_co['企业名称'].apply(clean_fullname).apply(clean_short_name).replace('',pd.NA)
patent_co['技术大类']=patent_co.IPC主分类.str.strip().str[:3]
patent_co['引证专利_A'] = patent_co['引证专利'].fillna('').str.split(';').apply(lambda x: [i.strip() for i in x if i.strip()])
patent_co=patent_co[patent_co['企业简称'].isin(combined_tax_gx_companies)].sort_values('申请年').reset_index(drop=True)
patent_co.shape,patent_co['企业简称'].nunique(),patent_co.申请号.nunique()

((1306185, 37), 8451, 1077022)

In [22]:
(19-4+1)*8451

135216

In [25]:
n = [6, 5, 4, 3, 2] 
m1 = [5, 4, 3, 2, 1] 
m2 = [4, 3, 2, 1, 0]
tuple_list =[]
for ni in N:
    tuple_list.append((ni, m1[n.index(ni)])) 
    tuple_list.append((ni, m2[n.index(ni)])) 
tuple_list

[(6, 5),
 (6, 4),
 (5, 4),
 (5, 3),
 (4, 3),
 (4, 2),
 (3, 2),
 (3, 1),
 (2, 1),
 (2, 0)]

In [ ]:
%%time
knowledge_year=5
df_res=pd.DataFrame()

for year in range(2004,2019+1):
    year_res=[]
    patent_value=patent_co[patent_co['申请年'] <=  year]
    max_date=pd.to_datetime(year, format='%Y')+ pd.offsets.YearEnd()

    for co in sorted(combined_tax_gx_companies):
        filter_patent= patent_value[patent_value['企业简称']==co]
        if filter_patent.empty:
            continue

        co_metrics ={'year':year,'企业简称':co,'qymc':map_name_co.get(co),'专利申请人':';'.join(filter_patent['企业名称'].dropna().unique())}
        co_metrics['知识宽度'] = process_ipc(filter_patent) # 累积量
        mask_years = filter_patent['申请年'].between(year - knowledge_year, year - 1)
        co_metrics['patent_width'] = process_ipc(filter_patent[mask_years]) 
        tech_category =  set(filter_patent.loc[mask_years,'技术大类'].dropna().to_list())
        past_citations = set(filter_patent.loc[mask_years, '引证专利_A'].explode().dropna())#.str.strip()
        past_patents = set(filter_patent.loc[filter_patent['申请年'] != year, '公开（公告）号'].dropna().str.strip())
        existing_knowledge = past_patents | past_citations

        for m,n in tuple_list:
            suffix=f'{m,n}'
            M=m*365
            N=n*365
            
            filter_value_date=filter_patent[filter_patent.申请日>=max_date-pd.Timedelta(days=N)]
            if filter_value_date.empty:
                continue
                
            mask=filter_value_date.申请日 <  max_date-pd.Timedelta(days=(N - M))
            tech_category_value = filter_value_date[mask]
            base_category_value = filter_value_date[~mask]
            
            innovation_patent_num=len(set(tech_category_value.技术大类)^set(base_category_value.技术大类)) 
            co_metrics[f'innovation_patent_num_ipc_{suffix}']=innovation_patent_num
            co_metrics[f'ipc_innovationc_{suffix}']=innovation_patent_num / len(filter_value_date)
        
        filter_value = filter_patent[filter_patent['申请年'] ==  year]
        if not filter_value.empty:           
            co_metrics.update(calculate_metrics(filter_value))   # 计算指标
       
            explorative_percentage = filter_value['引证专利_A'].apply(lambda x:len(set(x) & set(existing_knowledge)) / len(x) if (isinstance(x, list) and len(x)) else 0)
            co_metrics['explorative'] = (explorative_percentage > 0.8).sum() / len(explorative_percentage)
            co_metrics['jianjin_patent_num'] = filter_value['技术大类'].isin(tech_category).sum()
            
        year_res.append(co_metrics)

    df_res = pd.concat([df_res, pd.DataFrame(year_res)], ignore_index=True) 

    print(year,len(year_res),patent_value['企业简称'].nunique(),df_res.shape)


2004 786 786 (786, 57)
2005 1045 1045 (1831, 57)


In [102]:
def patent_width(row,year=5):
    filter_patent=patent_co[patent_co['申请年'].between(row.year - year, row.year)&(patent_co['企业简称'] == row['企业简称'])]
    return process_ipc(filter_patent)
    
#df_res['patent_width']=df_res.apply(patent_width, axis=1)
df_res

,year,企业简称,qymc,专利申请人,知识宽度,patent_width,tech_category,existing_knowledge,"innovation_patent_num_ipc_(6, 5)","ipc_innovationc_(6, 5)",...,waiguan,copatent_invention,copatent_shiyong,copatent_waiguan,"innovation_patent_num_ipc_(2, 0)","ipc_innovationc_(2, 0)",申请年_x,explorative,申请年_y,jianjin_patent_num
0,2004,UT斯达康通讯,UT斯达康通讯有限公司,UT斯达康通讯有限公司,0.000000,1.000000,,,1.0,0.250000,...,0.0,0.0,0.0,0.0,NaN,NaN,2004.0,0.0,2004.0,0.0
1,2004,万达信息,万达信息股份有限公司,万达信息股份有限公司,0.444444,0.444444,G06;H04,CN100411380C;CN1201190A;CN1210409A;CN1217265C;...,2.0,0.333333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2004,上海万兴生物制药,上海万兴生物制药有限公司,上海万兴生物制药有限公司,0.447347,0.447347,C12,CN100335622C;CN100339478C;CN1182155C;CN1194308...,1.0,0.250000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2004,上海东升新材料,上海东升新材料有限公司,上海东升新材料有限公司,0.320000,1.000000,,,1.0,1.000000,...,0.0,1.0,0.0,0.0,NaN,NaN,2004.0,0.0,2004.0,0.0
4,2004,上海东富龙科技,上海东富龙科技股份有限公司;上海东富龙科技有限公司,上海东富龙科技有限公司,0.493827,0.375000,F16;F26,CN1327149C;CN1480670A;CN2146614Y;CN2148219Y;CN...,3.0,0.500000,...,0.0,0.0,0.0,0.0,NaN,NaN,2004.0,0.0,2004.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74517,2019,龙建路桥,龙建路桥股份有限公司,龙建路桥股份有限公司,0.856684,0.863772,A01;B07;B09;B28;B65;E01;E02;E04;E21;G01,CN101033597B;CN101581065B;CN101586336A;CN10159...,13.0,0.361111,...,0.0,5.0,6.0,1.0,NaN,NaN,2019.0,0.0,2019.0,9.0
74518,2019,龙星化工,龙星化工股份有限公司,龙星化工股份有限公司,0.860021,0.774691,B01;B03;B65;C01;C09;F17;F23;G05,CN101279738A;CN101891976A;CN102286221B;CN10297...,8.0,0.571429,...,0.0,0.0,0.0,0.0,NaN,NaN,2019.0,0.0,2019.0,2.0
74519,2019,龙福环能科技,龙福环能科技股份有限公司,龙福环能科技股份有限公司,0.780104,0.822936,A23;A47;B08;B32;B65;D01;D02;D06;F24,CN100999845A;CN101025040A;CN101302671A;CN10130...,10.0,0.357143,...,0.0,0.0,0.0,0.0,NaN,NaN,2019.0,0.0,2019.0,5.0
74520,2019,龙芯中科技术,龙芯中科技术有限公司,龙芯中科技术有限公司;龙芯中科技术股份有限公司;龙芯中科(山西)技术有限公司;龙芯中科(成都...,0.398258,0.407921,G01;G05;G06;G09;G11;H01;H03;H04;H05,AU5538686A;CA2439633A1;CN100354851C;CN10057824...,10.0,0.024155,...,0.0,0.0,0.0,0.0,1.0,0.25,2019.0,0.0,2019.0,67.0


In [101]:
patent_co.to_csv('data\patent_co_tax_gx_1306_8k_2019.csv',encoding='utf-8')

In [97]:
df_res.columns

Index(['year', '企业简称', 'qymc', '专利申请人', '知识宽度', 'patent_width',
       'tech_category', 'existing_knowledge',
       'innovation_patent_num_ipc_(6, 5)', 'ipc_innovationc_(6, 5)',
       'innovation_patent_num_ipc_(6, 4)', 'ipc_innovationc_(6, 4)',
       'innovation_patent_num_ipc_(5, 4)', 'ipc_innovationc_(5, 4)',
       'innovation_patent_num_ipc_(5, 3)', 'ipc_innovationc_(5, 3)',
       'innovation_patent_num_ipc_(4, 3)', 'ipc_innovationc_(4, 3)',
       'innovation_patent_num_ipc_(4, 2)', 'ipc_innovationc_(4, 2)',
       'innovation_patent_num_ipc_(3, 2)', 'ipc_innovationc_(3, 2)',
       'innovation_patent_num_ipc_(3, 1)', 'ipc_innovationc_(3, 1)',
       'innovation_patent_num_ipc_(2, 1)', 'ipc_innovationc_(2, 1)',
       'Application_num', 'IPC主分类_去重', 'IPC主分类_平均', 'IPC_个数_去重', 'IPC_个数_平均',
       '被引证次数', '引证数量', '引证数量_去重', '引证数量_平均', '简单同族个数', '简单同族_去重',
       '简单同族美日欧_比例', '发明人数量', '发明人数量_去重', '发明人数量_平均', '权利要求数量', '引证科技文献数量',
       '有效数量', '有效_比例', '发明授权', '发明申请', '实用新型', 

In [103]:
del df_res['申请年_x']
del df_res['申请年_y']

In [65]:
qymc_gx_co=pd.read_excel('data\\税调&高新企业的税调的qymc.xlsx',index_col=0)
qymc_gx_co.shape

(109708, 2)

In [111]:
map_name_co=qymc_gx_co.groupby('企业简称')['qymc'].apply(lambda x:';'.join(set(x)))
# df_res['qymc']=df_res['企业简称'].map(map_name_co)
count_name_co=qymc_gx_co.groupby('企业简称')['qymc'].nunique().sort_values()
count_name_co.tail(20)

企业简称
SEW传动设备     4
重庆钢铁        4
今麦郎饮品       4
光宝电子        4
液化空气        4
建滔化工        4
光大环保能源      4
泰科电子        4
华锐风电科技      5
可口可乐饮料      5
泰山石膏        5
永丰余纸业       5
金红叶纸业       5
维达纸业        5
华润水泥        8
科技          9
燕京啤酒       10
青岛啤酒       11
华新水泥       15
华润雪花啤酒     25
Name: qymc, dtype: int64

In [106]:
(count_name_co>1).sum()

2666

In [107]:
2666/106699

0.024986176065380183

In [98]:
qymc_gx_co[qymc_gx_co.企业简称=='科技']

,qymc,企业简称
112138,中博龙辉北京)科技有限公司,科技
370243,华美财富北京)科技有限公司,科技
668173,安富利中国)科技有限公司,科技
764075,帮帮在线北京)科技有限公司,科技
882643,慧都创新北京)科技有限公司,科技
1176617,泰格斯通大连)科技有限公司,科技
1293355,清源华动北京)科技有限公司,科技
1508146,英视澳华北京)科技有限公司,科技
1566539,诺和诺德天津)科技有限公司,科技


In [113]:
df_res.loc[df_res['企业简称']=='科技',['企业简称','qymc','专利申请人']]

,企业简称,qymc,专利申请人


In [66]:
qymc_gx_co[qymc_gx_co.企业简称=='华新水泥']

,qymc,企业简称
368621,华新水泥(宜昌)有限公司,华新水泥
368622,华新水泥(岳阳)有限公司,华新水泥
368623,华新水泥(房县)有限公司,华新水泥
368624,华新水泥(昆明东川)有限公司,华新水泥
368625,华新水泥(昭通)有限公司,华新水泥
368626,华新水泥(株洲)有限公司,华新水泥
368627,华新水泥(桑植)有限公司,华新水泥
368628,华新水泥(秭归)有限公司,华新水泥
368629,华新水泥(迪庆)有限公司,华新水泥
368630,华新水泥(道县)有限公司,华新水泥


In [115]:
df_res.loc[df_res['企业简称']=='华新水泥',['企业简称','qymc','专利申请人']]

,企业简称,qymc,专利申请人
206,华新水泥,华新水泥(迪庆)有限公司;华新水泥(道县)有限公司;华新水泥(昭通)有限公司;华新水泥(长阳...,华新水泥股份有限公司
1064,华新水泥,华新水泥(迪庆)有限公司;华新水泥(道县)有限公司;华新水泥(昭通)有限公司;华新水泥(长阳...,华新水泥股份有限公司
2200,华新水泥,华新水泥(迪庆)有限公司;华新水泥(道县)有限公司;华新水泥(昭通)有限公司;华新水泥(长阳...,华新水泥股份有限公司
3675,华新水泥,华新水泥(迪庆)有限公司;华新水泥(道县)有限公司;华新水泥(昭通)有限公司;华新水泥(长阳...,华新水泥股份有限公司
5603,华新水泥,华新水泥(迪庆)有限公司;华新水泥(道县)有限公司;华新水泥(昭通)有限公司;华新水泥(长阳...,华新水泥股份有限公司
8161,华新水泥,华新水泥(迪庆)有限公司;华新水泥(道县)有限公司;华新水泥(昭通)有限公司;华新水泥(长阳...,华新水泥股份有限公司
11386,华新水泥,华新水泥(迪庆)有限公司;华新水泥(道县)有限公司;华新水泥(昭通)有限公司;华新水泥(长阳...,华新水泥股份有限公司
15333,华新水泥,华新水泥(迪庆)有限公司;华新水泥(道县)有限公司;华新水泥(昭通)有限公司;华新水泥(长阳...,华新水泥股份有限公司
19977,华新水泥,华新水泥(迪庆)有限公司;华新水泥(道县)有限公司;华新水泥(昭通)有限公司;华新水泥(长阳...,华新水泥股份有限公司
25286,华新水泥,华新水泥(迪庆)有限公司;华新水泥(道县)有限公司;华新水泥(昭通)有限公司;华新水泥(长阳...,华新水泥股份有限公司


In [72]:
patent_co.loc[patent_co.企业简称=='华新水泥',['企业简称','企业名称','申请人']].drop_duplicates()

,企业简称,企业名称,申请人
2027,华新水泥,华新水泥股份有限公司,华新水泥股份有限公司
351110,华新水泥,华新水泥股份有限公司,华新水泥股份有限公司; 华新环境工程有限公司; 华新水泥(黄石)装备制造有限公司
351156,华新水泥,华新水泥股份有限公司,华新环境工程有限公司; 华新水泥股份有限公司; 华新水泥(黄石)装备制造有限公司
351465,华新水泥,华新水泥股份有限公司,华新水泥(黄石)装备制造有限公司; 华新水泥股份有限公司; 华新环境工程有限公司
368436,华新水泥,华新水泥股份有限公司,华新环境工程有限公司; 华新水泥股份有限公司
737485,华新水泥,华新水泥股份有限公司,华新水泥股份有限公司; 华新水泥(黄石)装备制造有限公司
850326,华新水泥,华新水泥股份有限公司,华新水泥(黄石)装备制造有限公司; 华新水泥股份有限公司
926818,华新水泥,华新水泥股份有限公司,华新水泥股份有限公司; 华新环境工程有限公司
979274,华新水泥,华新水泥股份有限公司,华新新型建材(武汉)有限公司; 华新水泥股份有限公司
1004122,华新水泥,华新水泥股份有限公司,重庆齿轮箱有限责任公司; 华新水泥股份有限公司


In [99]:
df_res.to_excel('data\\tax_gx_year.xlsx')